In [ ]:
import wfcommons.wfchef.utils as utils

from QHyper.problems.algorithms.utils import wfworkflow_to_qhyper_workflow, merge_subworkflows
from QHyper.problems.algorithms.workflow_decomposition import HeftBasedAlgorithm
from QHyper.problems.workflow_scheduling import Workflow, WorkflowSchedulingProblem
from QHyper.solvers.gurobi import Gurobi

## Load and divide workflow

In [ ]:
tasks_file = "workflows_data/workflows/complex_workflow.json"
machines_file = "workflows_data/machines/machines_for_3_tasks_3_machines_1_path.json"
deadline = 50

In [ ]:
workflow = Workflow(tasks_file, machines_file, deadline)
utils.draw(workflow.wf_instance.workflow, with_labels=True)

In [ ]:
wf_workflow1, wf_workflow2 = HeftBasedAlgorithm().decompose(workflow, 2)
merged = merge_subworkflows([wf_workflow1, wf_workflow2])
utils.draw(merged, with_labels=True)

In [ ]:
workflow1 = wfworkflow_to_qhyper_workflow(wf_workflow1, machines_file, deadline / 2)
workflow2 = wfworkflow_to_qhyper_workflow(wf_workflow2, machines_file, deadline / 2)
wsp1 = WorkflowSchedulingProblem(workflow1)
wsp2 = WorkflowSchedulingProblem(workflow2)

## Solve the Workflow Schedluling Problem with the Gurobi solver.

In [ ]:
gurobi1 = Gurobi(problem=wsp1)
gurobi2 = Gurobi(problem=wsp2)

solution_gurobi1 = gurobi1.solve(params_inits={"name": "wsp1"})
solution_gurobi2 = gurobi2.solve(params_inits={"name": "wsp2"})

In [ ]:
solution1 = wsp1.decode_solution(solution_gurobi1)
solution2 = wsp2.decode_solution(solution_gurobi2)

print(f"Task assignment for first subgraph: {solution1}")
print(f"Task assignment for second subgraph: {solution2}")

In [ ]:
cost1 = wsp1.calculate_solution_cost(solution1)
time1 = wsp2.calculate_solution_timespan(solution1)
print(f"Cost of running first subworkflow: {cost1}")
print(f"Time of running first subworkflow: {time1}")

cost2 = wsp2.calculate_solution_cost(solution2)
time2 = wsp2.calculate_solution_timespan(solution2)
print(f"Cost of running second subworkflow: {cost2}")
print(f"Time of running second subworkflow: {time2}")

print(f"Cost of running full workflow: {cost1 + cost2}")
print(f"Time of running full workflow: {time1 + time2}")

## Version without split on Gurobi

In [ ]:
wsp = WorkflowSchedulingProblem(workflow)
gurobi = Gurobi(problem=wsp)
solution_gurobi = gurobi.solve(params_inits = {"name": "wsp"})

In [ ]:
solution = wsp.decode_solution(solution_gurobi)
print(f"Task assignment for full graph: {solution}")

In [ ]:
print(f"Cost of running full workflow: {wsp.calculate_solution_cost(solution)}")
print(f"Time of running full workflow: {wsp.calculate_solution_timespan(solution)}")